In [1]:
!uv run python --version

Python 3.12.10


In [3]:
import pickle

import catppuccin
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import matplotlib as mpl
import matplotlib.pyplot as plt
import mlflow
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

In [4]:
mpl.style.use(catppuccin.PALETTE.mocha.identifier)
%matplotlib inline

In [5]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/12/13 00:33:34 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/13 00:33:34 INFO mlflow.store.db.utils: Updating database tables
2025/12/13 00:33:34 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/13 00:33:34 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/13 00:33:34 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/13 00:33:34 INFO alembic.runtime.migration: Will assume non-transactional DDL.


<Experiment: artifact_location='/home/ludo/Public/Projets_IA/MLOps_ZoomCamp/02-experiment-tracking/mlruns/1', creation_time=1765564082300, experiment_id='1', last_update_time=1765564082300, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [6]:
# Jan 2021
# january2021 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet'
january2021 = './data/green_tripdata_2021-01.parquet'

# Feb 2021
# february2021 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet'
february2021 = './data/green_tripdata_2021-02.parquet'

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    mask = (df.duration >= 1) & (df.duration <= 60)
    df = df.loc[mask].assign(
        PULocationID=lambda d: d['PULocationID'].astype('string'),
        DOLocationID=lambda d: d['DOLocationID'].astype('string')
    )

    return df

In [8]:
df_train = read_dataframe(january2021)
df_val = read_dataframe(february2021)

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73908 entries, 0 to 76517
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               73908 non-null  int64         
 1   lpep_pickup_datetime   73908 non-null  datetime64[us]
 2   lpep_dropoff_datetime  73908 non-null  datetime64[us]
 3   store_and_fwd_flag     38175 non-null  object        
 4   RatecodeID             38175 non-null  float64       
 5   PULocationID           73908 non-null  string        
 6   DOLocationID           73908 non-null  string        
 7   passenger_count        38175 non-null  float64       
 8   trip_distance          73908 non-null  float64       
 9   fare_amount            73908 non-null  float64       
 10  extra                  73908 non-null  float64       
 11  mta_tax                73908 non-null  float64       
 12  tip_amount             73908 non-null  float64       
 13  tolls_

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)

# root_mean_squared_error(y_val, y_pred)

In [15]:
# fig, ax = plt.subplots(figsize=(8, 6))
# sns.histplot(y_pred, label='prediction', kde=True, ax=ax)
# sns.histplot(y_train, label='actual', kde=True, ax=ax)
# ax.legend()
# plt.show()

In [16]:
# with open('models/lin_reg.bin', 'wb') as f_out:
#     pickle.dump((dv, lr), f_out)

In [17]:
with mlflow.start_run():
    alpha = 0.1

    mlflow.set_tag("developper", "ludo")
    mlflow.log_param("train-data-path", january2021)
    mlflow.log_param("valid-data-path", february2021)

    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    print(f"{lr} → {rmse=:.2f}")

Lasso(alpha=0.1) → rmse=12.14


In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

In [21]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|             | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/ludo/Public/Projets_IA/MLOps_ZoomCamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:45:17] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.41766                             
[1]	validation-rmse:10.72351                             
[2]	validation-rmse:10.11951                             
[3]	validation-rmse:9.59665                              
[4]	validation-rmse:9.14517                              
[5]	validation-rmse:8.75739                              
[6]	validation-rmse:8.42506                              
[7]	validation-rmse:8.14107                              
[8]	validation-rmse:7.89856                              
[9]	validation-rmse:7.69336                              
[10]	validation-rmse:7.51825                             
[11]	validation-rmse:7.37004                             
[12]	validation-rmse:7.24550                             
[13]	validation-rmse:7.13984                             
[14]	validation-rmse:7.04984                             
[15]	validation-rmse:6.97413                             
[16]	validation-rmse:6.90959                             
[17]	validatio

/home/ludo/Public/Projets_IA/MLOps_ZoomCamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [01:46:05] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.74506                             
[1]	validation-rmse:11.31157                             
[2]	validation-rmse:10.91035                             
[3]	validation-rmse:10.53954                             
[4]	validation-rmse:10.19724                             
[5]	validation-rmse:9.88141                              
[6]	validation-rmse:9.59084                              
[7]	validation-rmse:9.32404                              
[8]	validation-rmse:9.07873                              
[9]	validation-rmse:8.85367                              
[10]	validation-rmse:8.64737                             
[11]	validation-rmse:8.45849                             
[12]	validation-rmse:8.28606                             
[13]	validation-rmse:8.12802                             
[14]	validation-rmse:7.98392                             
[15]	validation-rmse:7.85291                             
[16]	validation-rmse:7.73327                             
[17]	validatio

...

[0]	validation-rmse:11.67549                             
[1]	validation-rmse:11.18724                             
[2]	validation-rmse:10.74100                             
[3]	validation-rmse:10.33510                             
[4]	validation-rmse:9.96620                              
[5]	validation-rmse:9.63013                              
[6]	validation-rmse:9.32671                              
[7]	validation-rmse:9.05140                              
[8]	validation-rmse:8.80264                              
[9]	validation-rmse:8.58103                              
[10]	validation-rmse:8.37338                             
[11]	validation-rmse:8.19173                             
[12]	validation-rmse:8.02718                             
[13]	validation-rmse:7.87845                             
[14]	validation-rmse:7.74042                             
[15]	validation-rmse:7.62139                             
[16]	validation-rmse:7.51294                             
[17]	validatio

/home/ludo/Public/Projets_IA/MLOps_ZoomCamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [02:06:18] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.54332                             
[1]	validation-rmse:10.94356                             
[2]	validation-rmse:10.40905                             
[3]	validation-rmse:9.93253                              
[4]	validation-rmse:9.50870                              
[5]	validation-rmse:9.13428                              
[6]	validation-rmse:8.80267                              
[7]	validation-rmse:8.51003                              
[8]	validation-rmse:8.25245                              
[9]	validation-rmse:8.02896                              
[10]	validation-rmse:7.82960                             
[11]	validation-rmse:7.65613                             
[12]	validation-rmse:7.50634                             
[13]	validation-rmse:7.37296                             
[14]	validation-rmse:7.25700                             
[15]	validation-rmse:7.15682                             
[16]	validation-rmse:7.06920                             
[17]	validatio

/home/ludo/Public/Projets_IA/MLOps_ZoomCamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [02:06:56] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.39101                             
[1]	validation-rmse:10.68071                             
[2]	validation-rmse:10.06618                             
[3]	validation-rmse:9.54043                              
[4]	validation-rmse:9.08768                              
[5]	validation-rmse:8.70427                              
[6]	validation-rmse:8.37338                              
[7]	validation-rmse:8.09645                              
[8]	validation-rmse:7.85950                              
[9]	validation-rmse:7.66603                              
[10]	validation-rmse:7.50139                             
[11]	validation-rmse:7.35937                             
[12]	validation-rmse:7.24127                             
[13]	validation-rmse:7.13996                             
[14]	validation-rmse:7.05615                             
[15]	validation-rmse:6.98318                             
[16]	validation-rmse:6.92601                             
[17]	validatio

In [23]:
best_params = dict(
    learning_rate=0.11003124365651852,
    max_depth=69,
    min_child_weight=1.3037172702760245,
    objective="reg:linear",
    reg_alpha=0.06605109955963756,
    reg_lambda=0.34862327720948777,
    seed=42
)

In [24]:
mlflow.xgboost.autolog()
booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50,
)

2025/12/13 15:36:15 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '06af7ea8c07544a5a9366282afe7bd62', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/ludo/Public/Projets_IA/MLOps_ZoomCamp/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:36:16] WARNING: /workspace/src/objective/regression_obj.cu:282: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.31828
[1]	validation-rmse:10.54834
[2]	validation-rmse:9.89274
[3]	validation-rmse:9.33546
[4]	validation-rmse:8.86674
[5]	validation-rmse:8.46925
[6]	validation-rmse:8.13982
[7]	validation-rmse:7.86014
[8]	validation-rmse:7.63159
[9]	validation-rmse:7.43854
[10]	validation-rmse:7.27795
[11]	validation-rmse:7.14613
[12]	validation-rmse:7.03560
[13]	validation-rmse:6.94464
[14]	validation-rmse:6.86734
[15]	validation-rmse:6.80527
[16]	validation-rmse:6.75230
[17]	validation-rmse:6.70729
[18]	validation-rmse:6.67050
[19]	validation-rmse:6.63864
[20]	validation-rmse:6.61298
[21]	validation-rmse:6.59038
[22]	validation-rmse:6.56887
[23]	validation-rmse:6.55207
[24]	validation-rmse:6.53656
[25]	validation-rmse:6.52211
[26]	validation-rmse:6.51043
[27]	validation-rmse:6.50008
[28]	validation-rmse:6.49207
[29]	validation-rmse:6.48401
[30]	validation-rmse:6.47751
[31]	validation-rmse:6.47171
[32]	validation-rmse:6.46612
[33]	validation-rmse:6.46015
[34]	validation-rmse:6

2025/12/13 15:36:58 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/12/13 15:36:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/13 15:37:00 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
